In [3]:
example = """.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
..............."""

In [4]:
with open("./data/day7.txt") as f:
    data = f.read()

In [5]:
import numpy as np

def parse_input(input_str: str) -> np.array:
    lines = input_str.splitlines()
    grid = np.array([list(line) for line in lines])
    return grid

# Part 1

In [6]:
def update_tachyons(grid: np.array, spliters: set, beams: np.array) -> np.array:
    new_grid = grid.copy()
    rows, cols = grid.shape
    for beam in beams:
        new_grid[beam[0], beam[1]] = "/"
        if beam[0] + 1 >= rows:
            continue
        if grid[beam[0] + 1, beam[1]] == '^':
            spliters.add((beam[0] + 1, beam[1]))
            if beam[1] - 1 >= 0:
                new_grid[beam[0] + 1, beam[1] - 1] = "|"
            if beam[1] + 1 < cols:
                new_grid[beam[0] + 1, beam[1] + 1] = "|"
        elif grid[beam[0] + 1, beam[1]] == '.':
            new_grid[beam[0] + 1, beam[1]] = "|"
    return new_grid

def simulate_tachyons(grid: np.array, beams: np.array) -> np.array:
    spliters = set()
    while len(beams) > 0:
        grid = update_tachyons(grid, spliters, beams)
        beams = np.argwhere(grid == '|')
    return grid, spliters

In [7]:
grid = parse_input(example)
beams = np.argwhere(grid == 'S')
final_grid, spliters = simulate_tachyons(grid, beams)
print(final_grid)
print(len(spliters))


[['.' '.' '.' '.' '.' '.' '.' '/' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '/' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '/' '^' '/' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '/' '.' '/' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '/' '^' '/' '^' '/' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '/' '.' '/' '.' '/' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '/' '^' '/' '^' '/' '^' '/' '.' '.' '.' '.']
 ['.' '.' '.' '.' '/' '.' '/' '.' '/' '.' '/' '.' '.' '.' '.']
 ['.' '.' '.' '/' '^' '/' '^' '/' '/' '/' '^' '/' '.' '.' '.']
 ['.' '.' '.' '/' '.' '/' '.' '/' '/' '/' '.' '/' '.' '.' '.']
 ['.' '.' '/' '^' '/' '^' '/' '/' '/' '^' '/' '^' '/' '.' '.']
 ['.' '.' '/' '.' '/' '.' '/' '/' '/' '.' '/' '.' '/' '.' '.']
 ['.' '/' '^' '/' '/' '/' '^' '/' '/' '.' '/' '/' '^' '/' '.']
 ['.' '/' '.' '/' '/' '/' '.' '/' '/' '.' '/' '/' '.' '/' '.']
 ['/' '^' '/' '^' '/' '^' '/' '^' '/' '^' '/' '/' '/' '^' '/']
 ['/' '.' '/' '.' '/' '.' '/' '.' '/' '.' '/' '/' '/' '

In [8]:
grid = parse_input(data)
beams = np.argwhere(grid == "S")
final_grid, spliters = simulate_tachyons(grid, beams)
print(final_grid)
print(len(spliters))

[['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ['.' '.' '.' ... '.' '.' '.']
 ...
 ['.' '/' '.' ... '.' '/' '.']
 ['/' '^' '/' ... '/' '^' '/']
 ['/' '.' '/' ... '/' '.' '/']]
1640


# Part 2

In [54]:
def follow_beam(start_pos: np.array, grid: np.array) -> list[str]:
    final_paths = []
    rows, cols = grid.shape
    beam_positions = [(start_pos, "")]
    while len(beam_positions) > 0:
        current_pos, path = beam_positions.pop(0)
        if current_pos[0] + 1 >= rows:
            final_paths.append(path)
            continue
        if grid[current_pos[0] + 1, current_pos[1]] == '.':
            beam_positions.append((np.array([current_pos[0] + 1, current_pos[1]]), path))
        elif grid[current_pos[0] + 1, current_pos[1]] == '^':
            if current_pos[1] - 1 >= 0:
                beam_positions.append((np.array([current_pos[0] + 1, current_pos[1] - 1]), path + "L"))
            if current_pos[1] + 1 < cols:
                beam_positions.append((np.array([current_pos[0] + 1, current_pos[1] + 1]), path + "R"))
    return final_paths


def calculate_paths(grid: np.array) -> list[str]:
    start_positions = np.argwhere(grid == 'S')[0]
    rows, cols = grid.shape
    option_grid = np.zeros(grid.shape, dtype=int)
    new_grid = grid.copy()
    for i in range(int(start_positions[0]), rows):
        grid = new_grid.copy()
        for j in range(cols):
            if grid[i, j] == 'S':
                option_grid[i, j] = 1
            if grid[i, j] == '^' and (grid[i-1, j] == '|' or grid[i-1, j] == 'S'):
                if j - 1 >= 0:
                    option_grid[i, j - 1] += option_grid[i - 1, j]
                    new_grid[i, j - 1] = "|"
                if j + 1 < cols:
                    option_grid[i, j + 1] += option_grid[i - 1, j]
                    new_grid[i, j + 1] = "|"
            if grid[i, j] == '.' and (grid[i-1, j] == '|' or grid[i-1, j] == 'S'):
                option_grid[i, j] += option_grid[i-1, j]
                new_grid[i, j] = "|"
    return option_grid, new_grid

In [55]:
grid = parse_input(example)
option_grid, grid = calculate_paths( grid)
int(sum(option_grid[-1,:]))

40

In [56]:
grid = parse_input(data)
option_grid, grid = calculate_paths(grid)
int(sum(option_grid[-1, :]))

40999072541589

In [ ]:
example = """.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
..............."""

## SOME AI FUN

In [59]:
# Additional solver utilities: part1 (splits), part2 (timelines), and route enumeration/renderer
import numpy as np
from typing import Generator

def simulate_splits(grid: np.array) -> int:
    """Simulate classical beams and count splitter events.
    Beams start at `S` cells and move downward. When the cell below a beam is a '^',
    that beam is stopped and creates two beams at the same row as the splitter:
    (split_row, col-1) and (split_row, col+1).
    Returns total number of times a beam encountered a splitter.
    """
    rows, cols = grid.shape
    # start beam positions (row, col) as tuples
    beams = set(map(tuple, np.argwhere(grid == 'S')) )
    splits = 0
    while beams:
        next_beams = set()
        for r, c in beams:
            if r + 1 >= rows:
                # beam leaves the manifold
                continue
            below = grid[r + 1, c]
            if below == '^':
                # beam hits a splitter
                splits += 1
                if c - 1 >= 0:
                    next_beams.add((r + 1, c - 1))
                if c + 1 < cols:
                    next_beams.add((r + 1, c + 1))
            elif below == '.':
                # beam continues downward
                next_beams.add((r + 1, c))
            else:
                # treat any other cell (like 'S') as empty; continue downward
                next_beams.add((r + 1, c))
        beams = next_beams
    return splits

def count_timelines(grid: np.array) -> int:
    """Count total number of timelines for a single quantum particle.
    This counts each distinct timeline (each sequence of left/right choices),
    even if different timelines end at the same final cell.
    """
    rows, cols = grid.shape
    start = tuple(np.argwhere(grid == 'S')[0])
    # counts at current beam positions: mapping (r,c) -> number of timelines at that pos
    counts = {start: 1}
    total_leaves = 0
    while counts:
        next_counts = {}
        for (r, c), ways in counts.items():
            if r + 1 >= rows:
                # timeline(s) exit the manifold
                total_leaves += ways
                continue
            below = grid[r + 1, c]
            if below == '^':
                # split: propagate ways to left and right starting positions
                if c - 1 >= 0:
                    next_counts[(r + 1, c - 1)] = next_counts.get((r + 1, c - 1), 0) + ways
                if c + 1 < cols:
                    next_counts[(r + 1, c + 1)] = next_counts.get((r + 1, c + 1), 0) + ways
            elif below == '.':
                next_counts[(r + 1, c)] = next_counts.get((r + 1, c), 0) + ways
            else:
                next_counts[(r + 1, c)] = next_counts.get((r + 1, c), 0) + ways
        counts = next_counts
    return total_leaves

def enumerate_routes(grid: np.array, max_routes: int | None = None) -> Generator[str, None, None]:
    """Yield every possible route as a string of 'L'/'R' choices.
    Each character corresponds to the choice taken when a particle encounters a splitter.
    Use `max_routes` to limit enumeration for large inputs.
    """
    rows, cols = grid.shape
    start = tuple(np.argwhere(grid == 'S')[0])
    # stack for DFS: (position, path_string)
    stack = [(start, '')]
    yielded = 0
    while stack:
        (r, c), path = stack.pop()
        if r + 1 >= rows:
            yield path
            yielded += 1
            if max_routes is not None and yielded >= max_routes:
                return
            continue
        below = grid[r + 1, c]
        if below == '^':
            # push right then left so left is processed first (L is popped first)
            if c + 1 < cols:
                stack.append(((r + 1, c + 1), path + 'R'))
            if c - 1 >= 0:
                stack.append(((r + 1, c - 1), path + 'L'))
        else:
            # continue downward
            stack.append(((r + 1, c), path))

def render_route(grid: np.array, route: str) -> np.array:
    """Return a copy of the grid with a single route drawn using '|' characters.
    `route` is a string of 'L'/'R' choices taken at each splitter encountered.
    """
    g = grid.copy()
    rows, cols = g.shape
    r, c = tuple(np.argwhere(g == 'S')[0])
    idx = 0
    while r + 1 < rows:
        below = g[r + 1, c]
        if below == '^':
            if idx >= len(route):
                # route ended prematurely; stop
                break
            choice = route[idx]
            idx += 1
            # move to chosen adjacent cell on the splitter row and mark it
            if choice == 'L':
                c = c - 1
            else:
                c = c + 1
            r = r + 1
            if 0 <= c < cols:
                g[r, c] = '|'
        else:
            r = r + 1
            g[r, c] = '|'
    return g

def render_to_string(rendered_grid: np.array) -> str:
    return '\n'.join(''.join(row) for row in rendered_grid)

In [60]:
# Demo: run on the example and on the real input (from `data`)
grid_example = parse_input(example)
grid_input = parse_input(data)

print('Example: count splits (should match example) ->', simulate_splits(grid_example))
print('Example: number of timelines ->', count_timelines(grid_example))
# show some example routes (limit to first 10)
print('\nFirst 10 example routes (L/R strings):')
for i, route in enumerate(enumerate_routes(grid_example, max_routes=10), start=1):
    print(i, route)

# Now run on actual input (may be large)
part1 = simulate_splits(grid_input)
part2 = count_timelines(grid_input)
print('\nActual input: splits =', part1)
print('Actual input: timelines =', part2)

# If you want to inspect routes for the real input, you can iterate with a limit:
# for r in enumerate_routes(grid_input, max_routes=20):
#     print(r)

# Render a few routes from the example to ASCII for fun
print('\nRendering first 3 example routes:')
for n, route in enumerate(enumerate_routes(grid_example, max_routes=3), start=1):
    print(f'--- Route {n}:', route)
    print(render_to_string(render_route(grid_example, route)))
    print()

Example: count splits (should match example) -> 21
Example: number of timelines -> 40

First 10 example routes (L/R strings):
1 LLLLLLL
2 LLLLLLR
3 LLLLLRL
4 LLLLLRR
5 LLLLR
6 LLLRL
7 LLLRRLL
8 LLLRRLR
9 LLLRRRL
10 LLLRRRR

Actual input: splits = 1640
Actual input: timelines = 40999072541589

Rendering first 3 example routes:
--- Route 1: LLLLLLL
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
|^.^.^.^.^...^.
|..............

--- Route 2: LLLLLLR
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
.^|^.^.^.^...^.
..|............

--- Route 3: LLLLLRL
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........